# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** Classification. Teachers want to identify what groups of students need early intervention beofore they fail. An outcome should be at least two labels, good to go or need help. The labels may be better to have more than two. Among need help groups, some may need a little help, while others may need a great amount of help. The labels should be a good source for teachers to decide what actions they should take on what type of students.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
n_passed = student_data['passed'].value_counts()['yes']

# TODO: Calculate failing students
n_failed = student_data['passed'].value_counts()['no']

# TODO: Calculate graduation rate
grad_rate = n_passed.astype(float) / n_students * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train = None
X_test = None
y_train = None
y_test = None
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=300, random_state=42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
##### 1. K-Nearest Neighbors (KNeighbors)
- Real-world application<br/>
    Reference: [Introduction to the K-Nearest Neighbor (KNN) algorithm](http://www.datasciencecentral.com/profiles/blogs/introduction-to-the-k-nearest-neighbor-knn-algorithm)<br/>
    In the article, k-NN was applied to classify new automobile prototypes. A manufacturer wants to know what exisiting vehicles are close to currenlty develping new prototypes mostly for a marketing purpose. After finding nearest neighbors (similar type of vehicles), the manufacturer can learn what feature of the prototype is better than similar ones, which will be a good source of marketing. Like this example, k-NN can be applied to find the best group which the new data should belong to.


- Strengths<br/>
    Since k-NN is an instance based learning, there's no cost of training. The data will be categoraized instantly. Its simple algorithm is also a strength. Simply calculating distances between instacnes is all. There's no need to form decision boundary. The distance to the instacnes gives the result.


- Weaknesses<br/>
    k-NN must keep all raw data on memory to calculate the distances. Once the data size gets huge, computational time grows long. The curse of dimensionality is another problem to make computational time worse. Even though the data size is not huge, as a number of features goes bigger, it needs to calculate many distances. Also, as the number of features grows, the number of irrelevant features may grow. To use k-NN, it would be good to eliminate such irrelevant features.


- Why this model<br/>
    Teachers want to identify what groups of students tend to fail. From given dataset, it's possible to find that a new student is the closest to what groups based on similarity. If the similar students tend to fail, teachers can conclude the new student has a great possiblity to fail. Our test data doesn't have huge samples and features. k-NN won't cause any computational problem.
    

##### 2. Support Vector Machine (SVM)
- Real-world application<br/>
    Reference: [Application of support vector machine modeling for prediction of common diseases: the case of diabetes and pre-diabetes](http://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/1472-6947-10-16)<br/>
    This article discusses how SVM was applied to predict a person would have diabetes/pre-diabetes. People's medical records are parametrized and used as data set. SVM is a classifier to make margin wide as much as possible. The result, whether the person will have diabetes or not, shows SVM predicts well without any prior knowlege of data distribution. The healthcare is one of many other ares SVM can be applied. It is often said SVM is good in image processing area. (https://en.wikipedia.org/wiki/Support_vector_machine)


- Strengths<br/>
    The idea of SVM is to maximize the margin between decision boundary and closest data. For computation, only data close to the boundary (support vectors) are influential. Because of this, outliers doesn't bother SVM. During the margin maximization, SVM uses a regularizarion parameter. This has an effect to generalize the result and helps avoiding overfitting. Another strength is that the decision boundary on hyperplane can be linear or non-linear by SVM's kernel trick. Since the boundary will be formed on hyperplane, even though number of features gets big, SVM works effectively.


- Weaknesses<br/>
    When the linear boundry doesn't work, figuring out what kernel fits best would be a hard problem. This requires domain knowledge. In addition to choosing the best kernal, choosing the good parameters is another problem. Depending the parameters given, the result may overfit. Other than choosing good ones, as number of features grows in nonlinear kernel, computation will go expensive.


- Why this model<br/>
    The dataset may not be linealy separable into two categories, whether the groups of students tend to fail or not. In such a case, SVM will create a complex but effective decition boundary. The prediction accurary is expected to be high. The given dataset is not big, so the time for training won't be long.


##### 3. Logistic Regression
- Real-world application<br/>
    Reference: [Logistic Regression – Banking Case Study Example (Part 3)](http://ucanalytics.com/blogs/case-study-example-banking-logistic-regression-3/)<br/>
    The article is about banking service. The Logstic Regression was applied to predict a loan would be good or bad based on the age group. The result shows this binary classifier worked well for this kind of lineary separable dataset. As in the Wikipedia Logistic Regression webiste (https://en.wikipedia.org/wiki/Logistic_regression), this classifier is often used in banking, text processing and medial areas to categorize data.


- Strengths<br/>
    Logistic Regression is a simple algorithm and works well when given dataset can be linearly separable (only one decision boundary exists). The training is fast because of its simple algorithm. Like SVM, it also uses a regularization parameter which keeps the model from overfitting. Logistic Regression training is to find better parameters to form a better decision boundary. The parameters are independent from other features, so the caclulation can be done in parallel for each feature. This means the model doesn't have a problem of curse of diminsionality in terms of computational cost.


- Weaknesses<br/>
    Logistic Regression assumus only one decision boundary exists. The dataset should be linearly separable. If the model is complex and needs more than one decision boundaries, other classifier such as SVM would be the choice. Unlike SVM, Logistic Regression uses all training data equally. When irrelevant features come in, those will give unwated effects on the parameter calculation. In such a case, the prediction will get worse. Not just irrelevant features, outliers are the problam of Logis tic Regression since it tries to maximize likelihood using all training data.


- Why this model<br/>
    The given problem is a binary classification using many features. Teachers want to know what students will likely fail. Logistic Regsssion is the one always starting from for this kind of binary classification. It may give a good result thinking of the nature of the dataset.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [8]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import linear_model

# TODO: Initialize the three models
clf_A = KNeighborsClassifier()
clf_B = svm.SVC(random_state=0)
clf_C = linear_model.LogisticRegression(random_state=0)

# TODO: Set up the training set sizes
X_train_100 = None
y_train_100 = None

X_train_200 = None
y_train_200 = None

X_train_300 = None
y_train_300 = None

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
for clf in [clf_A, clf_B, clf_C]:
    for N in [100, 200, 300]:
        train_predict(clf, X_train[:N], y_train[:N], X_test, y_test)
        print '\n'
    print '\n'

Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0011 seconds
Made predictions in 0.0015 seconds.
F1 score for training set: 0.8060.
Made predictions in 0.0012 seconds.
F1 score for test set: 0.7246.


Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0006 seconds
Made predictions in 0.0025 seconds.
F1 score for training set: 0.8800.
Made predictions in 0.0015 seconds.
F1 score for test set: 0.7692.


Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0009 seconds
Made predictions in 0.0064 seconds.
F1 score for training set: 0.8809.
Made predictions in 0.0026 seconds.
F1 score for test set: 0.7801.




Training a SVC using a training set size of 100. . .
Trained model in 0.0017 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.8777.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.7746.


Training a SVC using a training set size of 200

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - K-Nearest Neighbors (KNeighbors)**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0011             |    0.0012              |    0.8060        |   0.7246        |
| 200               |      0.0006             |    0.0015              |    0.8800        |   0.7692        |
| 300               |      0.0009             |    0.0026              |    0.8809        |   0.7801        |

** Classifer 2 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0017               |    0.0008              |    0.8777        |    0.7746       |
| 200               |    0.0038               |    0.0012              |    0.8679        |    0.7815       |
| 300               |    0.0070               |    0.0016              |    0.8761        |    0.7838       |

** Classifer 3 - Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0013               |     0.0002             |    0.8593        |    0.7647       |
| 200               |    0.0025               |     0.0003             |    0.8562        |    0.7914       |
| 300               |    0.0031               |     0.0002             |    0.8468        |    0.8060       |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Look at the results of three different classifiers above. All three used the same training and test data. Each table shows how fast the training and prediction are, and how accurate the resutls are for both taining and test data. F1 Score ranges between 0 and 1.0. The best F1 Score is 1.0. If we add up Training Time and Prediction Time in test, Logistic Regression is the fastest. Logistic Regression has the best F1 Score, 0.8060, in test with 300 training set size. Logistic Regression looks a good choice. However, F1 Score in training goes down as the training size increases. If the model fits well, the accuracy for taining data should get better as the training size increases. Since the result was opposite, Logistic Regression may not be a good model for the data.

Let's compare K-Nearest Neighbors (k-NN) and Supprt Vector Machine (SVM). Upto 300 training data size, k-NN runs faster. However, from the nature of the algorithm, k-NN's peformance will certainly get worse. At some point, it will be worse than SVM once data size grows big. The prediction accuracies are similar, but SVM has better values overall. As training data size increases, the F1 Score of test data increases. The F1 Score of training data doesn't have an apparent increasing tendency, but at least, it doesn't decrease.

From this observation, SVM would be the best mode we should choose. SVM takes long to train; however, for 300 data, it takes 0.0099 seconds. Even though we need to deal with 100 times bigger data, still the training time would be withn bearable range. SVM's good accuracy will help teachers to correctly identify what students will fail.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Why Support Vector Machine (SVM) is expected to work well? There are two main reasons this algorithm would predict accurately. One is how a decision boundary is determined. The decision boundary devides training data in to two groups. For the sake of simplicity, let's assume only two features are involved in. This means training data can be plotted on a 2 demensional plane with labels (star or circle) as in the picture below. Also, for the sake of simplicity, let's assume training data can be devided by a straight line. To find the best line to devide training data in two groups, SVM tries to maximize the distances of the closest data points on the both side. The distance is called a margin. The closest data points are called support vectors. SVM's training goes like this: finds the closest pairs of star and circle, calculates the distances of each pair, draws a line that cuts throgh the middle of the each pair. The line should devide star and circle groups well since the line is on the farthest location to both groups. When test data are given, SVM checks the new data is on which side of the line. Lastly, SVM gives a label to each test data. This is a predicition.

![Margin of Support Vector Machine](https://www.analyticsvidhya.com/wp-content/uploads/2015/10/SVM_4.png)


The second one is how SVM solves a complex problem created by more than two deminsions and many features. In the real world data, a number of features are often more than two. When more than two features are involved in, the line will turn to a hyperplane. This is because features are expressed by multi-dimension data. SVM doesn't have any problem to form the hyperplane from support vectors which are multi-dimension data. In addition, the hyperplane, in another word, a decision boundary, should have a more complex shape rather than flat plane to devide groups. SVM has a nice feature called a kernel trick. Look at the picture blow. The picture shows an ellipse shaped decision boundary. In this case, a line never works as a decision boundary. The ellipse has a smooth line/surfece; however, a bumpy surface may work for some dataset. SVM still works well even in such a case becuase of the kernel trick. The shape of decision boundary is called a kernel. SVM has an ability to manage a complex kernel. Our data may or may not be separable by the straight line or plane. Whatever the shape of decision boundary is, SVM is expected to work well.

![Support Vector Machine Kernel Trick](https://www.analyticsvidhya.com/wp-content/uploads/2015/10/SVM_10.png)


From these reasons, SVM is expected to predict accuretly.


Note: Pictures are originally on the website, [Understanding Support Vector Machine algorithm from examples (along with code)](https://www.analyticsvidhya.com/blog/2015/10/understaing-support-vector-machine-example-code/)

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [9]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = [
    {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
     'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
     'kernel': ['rbf', 'sigmoid', 'poly'],
     'decision_function_shape': ['ovo', 'ovr', None],
     'tol':[1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
     }]

# TODO: Initialize the classifier
clf = svm.SVC(random_state=0)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print 'The best estimator\n', clf, '\n'

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

The best estimator
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.01,
  verbose=False) 

Made predictions in 0.0043 seconds.
Tuned model has a training F1 score of 0.8323.
Made predictions in 0.0015 seconds.
Tuned model has a testing F1 score of 0.7945.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** <br/>
Final F1 Score of SVM is:
- training: 0.8323
- testing: 0.7945

The untuned model got the result of 0.8761 for training and 0.7838 for testing. Training data score worsened, while testing data score improved. The reason of a worse score in training is, perhaps, overfitting was improved by tuning since testing data score is better.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.